In [ ]:
pip install pyomo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 3.6 MB/s eta 0:00:00


In [ ]:
pip install cbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 76

In [ ]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import matplotlib.pyplot as plt

In [ ]:
from pyomo.environ import *

In [ ]:
dataset = pd.read_csv("../data/input_wagner.csv", index_col=0)
dataset.head()

In [ ]:
months = np.arange(12, dtype=int) + 1
setup_cost = np.array([85, 102, 102, 101, 98, 114, 105, 86, 119, 110, 98, 114])
demand = np.array([69, 29, 36, 61, 61, 26, 34, 67, 45, 67, 79, 56])
inventory = np.ones(12)

dataset = pd.DataFrame(
    {"setup_cost": setup_cost, "inventory_cost": inventory, "demand": demand},
    index=months,
)

In [ ]:
model = pyo.ConcreteModel()

In [ ]:
model.T = pyo.Set(initialize=list(dataset.index))

In [ ]:
model.d = pyo.Param(model.T, initialize=dataset.demand)
model.s = pyo.Param(model.T, initialize=dataset.setup_cost)
model.h = pyo.Param(model.T, initialize=dataset.inventory_cost)

In [ ]:
model.x = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.y = pyo.Var(model.T, within=pyo.Binary)
model.I = pyo.Var(model.T, within=pyo.NonNegativeReals)

In [ ]:
def inventory_rule(model, t):
    if t == model.T.first():
        return model.I[t] == model.x[t] - model.d[t]
    else:
        return model.I[t] == model.I[model.T.prev(t)] + model.x[t] - model.d[t]

model.inventory_rule = pyo.Constraint(model.T, rule=inventory_rule)

In [ ]:
def get_max_antecip(t, dataset=dataset):
    """
    Returns the first instant in which it might be
    intelligent to produce a certain demand
    """

    total_inv = 0
    d = dataset.demand[t]
    s = dataset.setup_cost[t]
    out = t

    for i in range(1, t - dataset.index[0] + 1):
        h = dataset.inventory_cost[t - i]
        if total_inv + h * d <= s:
            total_inv = total_inv + h * d
            out = t - i
        else:
            break

    return out


def get_max_prod(t, dataset=dataset):
    df = dataset.query(f"max_antecip <= {t} & index >= {t}")
    return df.demand.sum()


dataset["max_antecip"] = [get_max_antecip(t, dataset=dataset) for t in dataset.index]
dataset["max_prod"] = [get_max_prod(t, dataset=dataset) for t in dataset.index]

In [ ]:
model.M = pyo.Param(model.T, initialize=dataset.max_prod)

def active_prod(model, t):
    return model.x[t] <= model.M[t] * model.y[t]

model.active_prod = pyo.Constraint(model.T, rule=active_prod)

In [ ]:
def total_cost(model):
    holding = sum(model.h[t] * model.I[t] for t in model.T)
    setup = sum(model.s[t] * model.y[t] for t in model.T)
    return setup + holding

model.obj = pyo.Objective(rule=total_cost, sense=pyo.minimize)

In [ ]:
solver = pyo.SolverFactory("cbc")

In [ ]:
solver.solve(model, tee=True)

ApplicationError: ignored

In [ ]:
# Obtain the maximum cost for comparison
max_cost = dataset.setup_cost.sum()
print(f"Maximum cost: {max_cost:.1f}")

In [ ]:
opt_value = model.obj()
print(f"Best cost {opt_value}")
print(f"% savings {100 * (1 - opt_value / max_cost) :.2f}")

In [ ]:
# Include production as a column
dataset["production"] = [model.x[t].value for t in dataset.index]

# Plot figure
fig, ax = plt.subplots(figsize=[6, 3], dpi=100)
x = dataset.index
width = 0.35
ax.bar(x - width/2, dataset.production, width, color="darkgreen", label="production")
ax.bar(x + width/2, dataset.demand, width, color="navy", label="demand")
ax.set_xticks(x)
ax.set_ylabel("Qtd")
ax.set_xlabel("t")
ax.legend()
fig.tight_layout()
plt.show()


In [ ]:
dataset["production"] = [model.x[t].value for t in dataset.index]
dataset["inventory"] = [model.I[t].value for t in dataset.index]
dataset[["demand", "production", "inventory"]]